<a href="https://colab.research.google.com/github/manjotmb20/Audio-Classification-using-MFCCs/blob/master/BGCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from numpy import linalg as LA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import nltk
nltk.download("punkt")
# Warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
data=pd.read_csv('data.csv')
data.head()

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [0]:
df_train = pd.read_csv('train.csv')
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train = df_train.drop(['Ticket', 'Cabin'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)
pd.crosstab(df_train['Title'], df_train['Sex'])
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Don', 'Sir', 'Jonkheer', 'Dona'],'Royalty')
    dataset['Title'] = dataset['Title'].replace(['Capt', 'Col','Dr','Major','Rev'],'Special')
df_train[['Title','Survived']].groupby(['Title'], as_index=False).mean()
title_mapping = {"Master": 1, "Miss": 2, "Mrs": 3, "Mr": 4, "Royalty": 5, "Special": 6}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
df_train = df_train.drop(['Name', 'PassengerId'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
median_age = np.zeros((2,3))
for dataset in combine:
    for sex in range(0,2):
        for pclass in range(0,3):
            guess_df = dataset[(dataset['Sex'] == sex) & \
                                  (dataset['Pclass'] == pclass+1)]['Age'].dropna()
            age_guess = guess_df.median()
            median_age[sex,pclass] = age_guess

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = median_age[i,j]

dataset['Age'] = dataset['Age'].astype(int)
df_train['AgeBand'] = pd.cut(df_train['Age'], 5)
df_train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)
for dataset in combine:
    dataset.loc[ dataset['Age'] <= 16, 'AgeG'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'AgeG'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'AgeG'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'AgeG'] = 3
    dataset.loc[ dataset['Age'] > 64, 'AgeG'] = 4
df_train = df_train.drop(['AgeBand','Age'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

df_train[['FamilySize','Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False) 
df_train = df_train.drop(['SibSp', 'Parch'], axis=1)
combine = [df_train]
freq_port = df_train.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
df_train[['Embarked','Survived']].groupby(['Embarked'], as_index = False).mean().sort_values(by='Survived', ascending=False)
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
df_train = df_train.drop(['Fare'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset.AgeG = dataset.AgeG.astype(int)
X_train = df_train.drop("Survived", axis=1)
Y_train = df_train['Survived'] 

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
classifier3=SVC(probability=True)
from sklearn.metrics import roc_auc_score,accuracy_score
x_train,x_val,y_train,y_val=train_test_split(X_train,Y_train,test_size=0.2,random_state=9)
classifier=LogisticRegression(random_state=0)
classifier.fit(x_train,y_train)
ypred=classifier.predict(x_val)
print("ROC score of LR",roc_auc_score(y_val,ypred))
y_prob=classifier.predict_proba(x_val)
df=pd.DataFrame()
df['pred']=ypred
survived=pd.get_dummies(df.pred,prefix='Survived')
df=pd.concat([df,survived],axis=1)
df.drop('pred',inplace=True,axis=1)
df2=pd.DataFrame()
classifier3.fit(x_train,y_train)
ypred3=classifier3.predict(x_val)
print("ROC score of SVM",roc_auc_score(y_val,ypred3))
df2['pred']=ypred3
y_prob3=classifier3.predict_proba(x_val)
survived2=pd.get_dummies(df2.pred,prefix='Surviveds')
df=pd.concat([df,survived2],axis=1)

ROC score of LR 0.7614556962025316
ROC score of SVM 0.7654430379746835


In [37]:
df

,Survived_0,Survived_1,Surviveds_0,Surviveds_1
0,1,0,0,1
1,1,0,1,0
2,1,0,1,0
3,1,0,1,0
4,0,1,0,1
...,...,...,...,...
174,0,1,0,1
175,0,1,0,1
176,1,0,1,0
177,0,1,0,1


In [38]:
A=np.array(df)
A.shape

(179, 4)

In [39]:
U=np.array(y_prob)
U.shape

(179, 2)

In [40]:
objclass_mat1=pd.DataFrame(y_prob,columns=['Survived','NotSurvived'])
objclass_mat2=pd.DataFrame(y_prob3,columns=['Survived','NotSurvived'])
df_t=pd.DataFrame(objclass_mat1)
Fo=np.array(df_t)
gpclass_mat=pd.DataFrame(columns=['Survived','NotSurvived'],index=['class1surv','class1notsurv','class2surv','class2notsurv'])
prob1s=np.sum(objclass_mat1.Survived)/len(objclass_mat1)
prob1ns=np.sum(objclass_mat1.NotSurvived)/len(objclass_mat1)
prob2s=np.sum(objclass_mat2.Survived)/len(objclass_mat2)
prob2ns=np.sum(objclass_mat2.NotSurvived)/len(objclass_mat2)
list1=[]
list1.append(prob1s)
list1.append(prob1ns)
list1.append(prob2s)
list1.append(prob2ns)

list2=[]
list2.append(prob1ns)
list2.append(prob1s)
list2.append(prob2ns)
list2.append(prob2s)
gpclass_mat['Survived']=list1
gpclass_mat['NotSurvived']=list2
gpclass_mat
Fg=np.array(gpclass_mat)
Fg

array([[0.62841452, 0.37158548],
       [0.37158548, 0.62841452],
       [0.61467118, 0.38532882],
       [0.38532882, 0.61467118]])

In [0]:
Q=np.array(Fg)

In [42]:
m=np.array(df)
Yg=np.zeros(shape=(m.shape[1],2))
c=np.sum(m,axis=0)
for i in range(Yg.shape[0]):
  if i%2==0:
    Yg[i][0]=c[i]/m.shape[0]
    Yg[i][1]=1-Yg[i][0]
  else:
    Yg[i][1]=c[i]/m.shape[0]
    Yg[i][0]=1-Yg[i][1]  
Yg.shape    

(4, 2)

In [0]:
Y=np.array(Yg)

In [0]:
import sys
threshold=sys.float_info.epsilon

In [45]:
D_sum=np.sum(A,axis=0)
Dv=np.diag(D_sum)
Dv

array([[118,   0,   0,   0],
       [  0,  61,   0,   0],
       [  0,   0, 112,   0],
       [  0,   0,   0,  67]], dtype=uint64)

In [0]:
Dn_sum=np.sum(A,axis=1)
Dn=np.diag(Dn_sum)

In [47]:
K_sum=np.sum(Y,axis=1)
K=np.diag(K_sum)
K

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [48]:
Ut_old=np.zeros(shape=U.shape)
alpha=0.40
Ut=U
while LA.norm((Ut-Ut_old))>threshold:
  Ut_old=Ut
  Q=np.matmul((np.linalg.inv(Dv+alpha*K)),((np.matmul(np.transpose(A),Ut_old))+alpha*np.matmul(K,Y)))
  t=np.matmul(A,Q)
  Ut=np.matmul(np.linalg.inv(Dn),t)
Ut.shape  

(179, 2)

In [0]:
from sklearn.metrics import accuracy_score
result=np.zeros(shape=Ut.shape[0])
for i in range(Ut.shape[0]):
  if Ut[i][0]<Ut[i][1]:
    result[i]=1

In [53]:
roc_auc_score(y_val,result)

0.5